In [1]:
print('Hackathon')

Hackathon


In [2]:
import pandas as pd
import numpy as np
from sklearn.metrics import roc_auc_score
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier
from sklearn.multioutput import MultiOutputClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split

In [3]:
training_set_features = pd.read_csv("training_set_features.csv")
training_set_labels = pd.read_csv("training_set_labels.csv")
test_set_features = pd.read_csv("test_set_features.csv")

In [4]:
training_set_features.set_index("respondent_id", inplace = True)
training_set_labels.set_index("respondent_id", inplace = True)
test_set_features.set_index("respondent_id", inplace = True)

In [5]:
training_set_features.head()

,xyz_concern,xyz_knowledge,behavioral_antiviral_meds,behavioral_avoidance,behavioral_face_mask,behavioral_wash_hands,behavioral_large_gatherings,behavioral_outside_home,behavioral_touch_face,doctor_recc_xyz,...,income_poverty,marital_status,rent_or_own,employment_status,hhs_geo_region,census_msa,household_adults,household_children,employment_industry,employment_occupation
respondent_id,,,,,,,,,,,,,,,,,,,,,
0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,...,Below Poverty,Not Married,Own,Not in Labor Force,oxchjgsf,Non-MSA,0.0,0.0,NaN,NaN
1,3.0,2.0,0.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,...,Below Poverty,Not Married,Rent,Employed,bhuqouqj,"MSA, Not Principle City",0.0,0.0,pxcmvdjn,xgwztkwe
2,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,NaN,...,"<= $75,000, Above Poverty",Not Married,Own,Employed,qufhixun,"MSA, Not Principle City",2.0,0.0,rucpziij,xtkaffoo
3,1.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,...,Below Poverty,Not Married,Rent,Not in Labor Force,lrircsnp,"MSA, Principle City",0.0,0.0,NaN,NaN
4,2.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,1.0,0.0,...,"<= $75,000, Above Poverty",Married,Own,Employed,qufhixun,"MSA, Not Principle City",1.0,0.0,wxleyezf,emcorrxb


In [6]:
training_set_labels.head()

,xyz_vaccine,seasonal_vaccine
respondent_id,,
0,0,0
1,0,1
2,0,0
3,0,1
4,0,0


In [7]:
test_set_features.head()

,xyz_concern,xyz_knowledge,behavioral_antiviral_meds,behavioral_avoidance,behavioral_face_mask,behavioral_wash_hands,behavioral_large_gatherings,behavioral_outside_home,behavioral_touch_face,doctor_recc_xyz,...,income_poverty,marital_status,rent_or_own,employment_status,hhs_geo_region,census_msa,household_adults,household_children,employment_industry,employment_occupation
respondent_id,,,,,,,,,,,,,,,,,,,,,
26707,2.0,2.0,0.0,1.0,0.0,1.0,1.0,0.0,1.0,0.0,...,"> $75,000",Not Married,Rent,Employed,mlyzmhmf,"MSA, Not Principle City",1.0,0.0,atmlpfrs,hfxkjkmi
26708,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,Below Poverty,Not Married,Rent,Employed,bhuqouqj,Non-MSA,3.0,0.0,atmlpfrs,xqwwgdyp
26709,2.0,2.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,0.0,...,"> $75,000",Married,Own,Employed,lrircsnp,Non-MSA,1.0,0.0,nduyfdeo,pvmttkik
26710,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,"<= $75,000, Above Poverty",Married,Own,Not in Labor Force,lrircsnp,"MSA, Not Principle City",1.0,0.0,NaN,NaN
26711,3.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,0.0,...,"<= $75,000, Above Poverty",Not Married,Own,Employed,lzgpxyit,Non-MSA,0.0,1.0,fcxhlnwr,mxkfnird


In [8]:
numeric_cols = ["xyz_concern", "xyz_knowledge", "behavioral_antiviral_meds", "behavioral_avoidance", "behavioral_face_mask", "behavioral_wash_hands", "behavioral_large_gatherings", "behavioral_outside_home", "behavioral_touch_face", "doctor_recc_xyz", "doctor_recc_seasonal", "chronic_med_condition", "child_under_6_months", "health_worker", "health_insurance", "opinion_xyz_vacc_effective", "opinion_xyz_risk", "opinion_xyz_sick_from_vacc", "opinion_seas_vacc_effective", "opinion_seas_risk", "opinion_seas_sick_from_vacc",  "household_adults", "household_children"]
non_numeric_cols = ["age_group", "education", "race", "sex", "income_poverty", "marital_status", "rent_or_own", "employment_status", "hhs_geo_region", "census_msa", "employment_industry", "employment_occupation"]
numeric = Pipeline(steps=[('imputer', SimpleImputer(strategy='mean')),('scaler', StandardScaler())])
non_numeric = Pipeline(steps=[('imputer', SimpleImputer(strategy='most_frequent')),('onehot', OneHotEncoder(handle_unknown='ignore', sparse = False))])
preprocessor = ColumnTransformer(transformers=[('num', numeric, numeric_cols), ('non_num', non_numeric, non_numeric_cols)], remainder = "passthrough")

In [9]:
X_train = training_set_features
y_train = training_set_labels
X_test = test_set_features
X_train_preprocessed = preprocessor.fit_transform(X_train)
X_test_preprocessed = preprocessor.fit_transform(X_test)

C:\Users\Arsh\anaconda3\Lib\site-packages\sklearn\preprocessing\_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(
C:\Users\Arsh\anaconda3\Lib\site-packages\sklearn\preprocessing\_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


In [10]:
X_train_split, X_split, y_train_split, y_split = train_test_split(X_train_preprocessed, y_train, test_size=0.09, random_state=39)
rf_model = RandomForestClassifier(n_estimators=300, random_state=39)
multi_target_model = MultiOutputClassifier(rf_model, n_jobs=-1)
multi_target_model.fit(X_train_split, y_train_split)
y_prediction = multi_target_model.predict_proba(X_split)
y_prediction_array = np.array(y_prediction)[:,:,1].T

In [11]:
xyz = roc_auc_score(y_split['xyz_vaccine'], y_prediction_array[:,0])
print('ROC AUC FOR XYZ VACCINE IS:', xyz)
seasonal = roc_auc_score(y_split['seasonal_vaccine'], y_prediction_array[:,1])
print('ROC AUC FOR SEASONAL VACCINE IS:', seasonal)

ROC AUC FOR XYZ VACCINE IS: 0.8737895400126025
ROC AUC FOR SEASONAL VACCINE IS: 0.8676149146507268


In [12]:
rf_model = RandomForestClassifier(n_estimators=300, random_state=39, n_jobs=2)
multi_target_model = MultiOutputClassifier(rf_model, n_jobs=2)
para_grid = {'estimator__n_estimators': [200, 300], 'estimator__max_depth': [None, 10], 'estimator__min_samples_split': [5, 7], 'estimator__min_samples_leaf': [1, 2]}
grid_search = GridSearchCV(multi_target_model, para_grid, cv=7, scoring='roc_auc', n_jobs=2)
grid_search.fit(X_train_split, y_train_split)
best_para = grid_search.best_params_
best_score = grid_search.best_score_
print('Best parameters are: ', best_para)
print('Best ROC AUC score from GridSearchCV is: ', best_score)

Best parameters are:  {'estimator__max_depth': None, 'estimator__min_samples_leaf': 2, 'estimator__min_samples_split': 7, 'estimator__n_estimators': 300}
Best ROC AUC score from GridSearchCV is:  0.859437717144137


In [13]:
best_model = grid_search.best_estimator_
y_prediction = best_model.predict_proba(X_test_preprocessed)
y_prediction_array = np.array(y_prediction)[:,:,1].T

In [14]:
final = pd.DataFrame({'respondent_id': test_set_features.index, 'xyz_vaccine': y_prediction_array[:,0], 'seasonal_vaccine': y_prediction_array[:,1]})
final.to_csv('final.csv', index=False)
final.head()

,respondent_id,xyz_vaccine,seasonal_vaccine
0,26707,0.201403,0.348373
1,26708,0.036599,0.071451
2,26709,0.254154,0.796093
3,26710,0.546867,0.877851
4,26711,0.319121,0.457861
